In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# <span style="color:red;"><center>TD_03_2 - RSA 1/2 mise en place des outils</center></span>

Le chiffrement RSA est nommé d'après les initiales de ses trois inventeurs Ronald Rivest, Adi Shamir et Leonard Adleman. C'est une méthode de chiffrement asymétrique inventée en 1977 et encore utilisée aujourd'hui pour le commerce électronique ou encore l'échange de données confidentielles sur internet.


Le TD se découpe en deux parties, une première pour mettre en places les différents outils nécessaires puis une seconde sur l'algorithme RSA spécifiquement.

## Exercice 1. Exponentiation rapide modulaire

1. Rappeler ici la fonction sur laquelle nous avons déjà travaillé qui prend trois arguments x, n et m et qui renvoie le résultat de $x^n (mod\ m)$. Cette fonctionne devra être récursive (pour s'entrainer) et utiliser $x^0 = 1$ et $x^n = ((x^2\ mod\ m)^{n//2})\ mod\ m$ si n est pair et $x^n = ((x\ mod\ m) \times (x^2\ mod\ m)^{n//2})\ mod\ m$ si n est impair.

In [4]:
# question 1
def expo(x,n,m):
    if n == 0:
        rep = 1
    
    rep = (((x**2%m)**(n//2))%m) if n%2==0 else (((x%m)*(((x**2)%m)**(n//2))%m))
    return rep

expo(2,10,100)

24

In [5]:
#correction
def puiss(x,n,m):
    if n == 0:
        return 1
    else:
        return ((x%m)**(n%2) * puiss((x*x)%m, n//2, m))%m

puiss(2., 10, 100)

24.0

## Exercice 2. Décomposer un texte en une liste de valeurs

Nous avons déjà travaillé sur les fonctions **chr** et **ord** 


On souhaite décomposer un texte sous forme d’une liste de valeurs numériques sur lesquelles travailler. On utilisera les valeurs de la table ASCII accessibles grâce à la fonction python ord(car) qui prend un seul caractère comme paramètre.

1. Ecrire une fonction **code(texte)** qui prend en entrée une chaine de caractères texte et qui renvoie une liste correspondant à la valeur de chaque caractère dans la table ASCII.
       
2. Ecrire une fonction **decode(tab)** qui effectue l’opération inverse. On pourra utiliser la fonction python chr(i) qui prend comme paramètre un entier et effectue l’opération inverse de ord(car).
       
3. Tester les fonctions obtenues à l’aide de phrases de votre choix

In [6]:
# Question 1.
def code(texte):
    assert type(texte) == str
    return [ord(car) for car in texte]


In [7]:
# Question 2.
def decode(tab):
    assert type(tab) == list
    return "".join(map(chr,tab))

In [8]:
# Question 3.
tab = code('HELLO mon name is Michel')
decodetab = decode(tab)

print(tab)
print(decodetab)


[72, 69, 76, 76, 79, 32, 109, 111, 110, 32, 110, 97, 109, 101, 32, 105, 115, 32, 77, 105, 99, 104, 101, 108]
HELLO mon name is Michel


## Exercice 3. Découpage du texte par blocs

Pour éviter une attaque statistique, il faut découper le message original en blocs. Ici, on se limites aux caractères de la table ASCII, chaque caractère se code donc sur trois chiffres (le caractère 65 sera donc codé 065). On collera ensuite tous les chiffres obtenus dans l’ordre puis on découpera par blocs de taille fixée (4 par exemple). On comblera le dernier bloc en ajoutant des 0.

exemple:
« Bonjour » donnerait [66,111,110,106,111,117,114]
en rajoutant les zéros pour coder chaque caractère sur trois chiffres [066, 111, 110, 106, 111, 117, 114]
puis on regroupe tout 066111110106111117114
Enfin on découpe par blocs de 4 et on complète le dernier bloc par des 0. [0661, 1111, 0106, 1111, 1711, 4000].


Il faudra passer par les chaines de caractères pour ne pas perdre les zéros en début de bloc. La liste finale renvoyée ne devra contenir que des entiers car nous réaliseront le chiffrement sur ces valeurs.

1. Ecrire une fonction **trois** qui prend comme argument un caractère et qui renvoie une chaine de trois caractères (exactement) correspondants au codage sur trois chiffres du caractère de départ.

2. Ecrire une fonction **regroupe** qui prend comme paramètre une chaine de caractères et qui renvoie la concaténation des différents blocs de 3 caractères obtenus.

3. Ecrire une fonction **decoupe** qui prend en paramètre une chaine de caractères et un entier n et qui renvoie une liste de chaines de n caractères correspondant au découpage par blocs après codage ascii de chaque caractère. Elle devra aussi compléter le dernier bloc avec des 0.

4. Proposer une fonction **valeurs** qui prend une chaine de caractères en entrée et qui renvoie la liste des entiers obtenus après découpage.

5. Proposer une fonction **texte** qui effectue l’opération inverse (vous pouvez proposer toutes les fonctions intermédiaires de votre choix)

In [9]:
bj = "Bonjour"

In [10]:
# Question 1.
def trois(char):
    char = ord(char)
    if len(str(char))== 1:
        return str("00" + str(char))
    elif len(str(char))== 2:
        return str("0" + str(char))
    else:
        return str(char)
trois("B")


'066'

In [11]:
def regroupe(chaine):
    return "".join([trois(char) for char in chaine])

regroupe(bj)

'066111110106111117114'

In [12]:
# Question 3.
def decoupe(chaine, n):
    chaine=regroupe(chaine)
    txt = [chaine[i:i+n] for i in range(0, len(chaine), n)]
    if len(txt[-1])!=n:
        txt[-1]+="0"*(n-len(txt[-1]))
    return txt
decoupe(bj,4)

['0661', '1111', '0106', '1111', '1711', '4000']

In [14]:
# Question 4.

#correction:

def valeurs(texte,n):
    return list(map(int,decoupe(texte,n)))

valeurs(bj, 4)

[661, 1111, 106, 1111, 1711, 4000]

In [16]:
# Question 5.
def complete(txt,n):
    while len(txt)<n:
        txt = '0'+txt
    return txt
    def texte(liste, n):
        liste = list(map(str,liste))
        liste = [complete(val,n) for val in liste]
        liste = "".join(liste)
        liste = [liste[i:i+3] for i in range(0,len(liste),3)]
        liste = list(map(int(liste)))
        while liste[-1] == 0:
            liste.pop()
        liste = list(map(chr,liste))
        return ''.join(liste)
    
tab = valeurs(bj, 4)
print(tab)
print(complete(tab,4))

[661, 1111, 106, 1111, 1711, 4000]
[661, 1111, 106, 1111, 1711, 4000]


## Exercice 4. Euclide et PGCD

1. Ecrire une fonction **choix_pq** qui renvoie deux nombres premiers choisis aléatoirement (parmi ceux générés au TD précédent par exemple).

2. Ecrire une fonction **Euclide** qui prend comme arguments deux entiers a et b et qui renvoie le quotient et le reste de la division Euclidienne de a par b.

3. Ecrire une fonction **PGCD** qui prend comme paramètres deux entiers a et b et qui renvoie leur PGCD (plus grand diviseur commun). Pour cela, on remarquera que PGCD(a, b) = PGCD(b, a%b)

4. Ecrire une fonction **Bezout** qui prend comme paramètres deux entiers a et b et qui renvoie les coefficients de l’égalité de Bézout c’est à dire un couple (u, v) tel que a*u + b*v = PGCD(a,b). Cette fonction peut aussi renvoyer le PGCD(a,b)
Pour cela on initialise u1 à 1, v1 à 0, u2 à 0 et v2 à 1
Puis on réalise la boucle :
  Soit q et r le quotient et le reste de la division euclidienne de a par b, on a donc:
  
  
  $a = q \times b + r$ soit $u_1 \times a + v_1 \times b = q \times (u_2 \times a + v2 \times b) + r$ et donc $(u_1 - q \times u_2) \times a + (v_1 - q \times v_2) \times b = r$
  
  
On va ensuite calculer le quotient et le reste de la division euclidienne de $(u_2 \times a + v_2 \times b)$ par r et ainsi de suite jusqu’à obtenir un reste nul.

Le dernier reste non nul obtenu est le PGCD(a,b), et les derniers coefficients $u_1 - q \times u_2$ et $v_1 - q \times v_2$ sont les deux coefficients de l’égalité de Bézout recherchés.

In [49]:
from math import *
def isPrime(n):
    prime = True
    i = 1
    while i < n:
        if n % i == 0 and i > 1:
            prime = False
        i += 1
    return True if prime else False

In [56]:
# Question 1.
from random import randint
def choix_pq():
    primelist = []
    while len(primelist) != 2:
        n = randint(0,100*10)
        if isPrime(n):
            primelist.append(n)
    return primelist

choix_pq()

[997, 199]

In [63]:
# Question 2.
def euclide(a,b):
    return a//b,a%b

euclide(6894,23)

(299, 17)

In [68]:
# Question 3.
def PGCD(a,b):
    assert a>=b
    if a%b ==0:
        return b
    else:
        return PGCD(b,a%b)

PGCD(9,3)
PGCD(1024,50)

3

2

In [66]:
# Question 4.
def bezout(a,b,u1=1,u2=0,v1=0,v2=1):
    assert a>=b
    if a%b == 0:
        return u2,v2
    else:
        return bezout(b,a%b,u2,u1-a//b*u2,v2,v1-a//b*v2)
bezout(101,97)

(-24, 25)

Tous les outils sont maintenant prêts, nous allons pouvoir commencer à explorer l'algorithme RSA!